In [10]:
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
import base64
from openai import OpenAI
import fitz  # PyMuPDF
from pathlib import Path

In [15]:
client = OpenAI()
downloads = Path("./../downloads")
pdfs = list((downloads / "pdfs").iterdir())
images = downloads / "images"
dishes = [p.name.replace(".pdf", "") for p in pdfs]

In [13]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def pdf2image(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    image_pth = images / pdf_path.name.replace(".pdf", "")
    image_pth.mkdir()
    # Iterate through each page and save it as an image
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        pix = page.get_pixmap()
        image_path = image_pth / f'page_{page_num + 1}.png'
        pix.save(image_path)

    pdf_document.close()

In [14]:
_ = [pdf2image(p) for p in pdfs]

In [21]:
prompt = """You are a helpful AI, your task is to read the recipe and note down all the ingredients for 1 person only in a CSV format with the following headers
dish_name,name, tags, quantity, metric
- quantity is always a number
- metric should be one of {tbsp, tsp, unit, g, cup}
- tags should be one of {Protein, Pasta, Vegetable, Sauce, Dairy, Spice, Herb, Grain}"""

In [22]:
dish = dishes[0]

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": prompt,
        }
      ] + [
        {
          "type": "image_url",
          "image_url": {
            "detail": "high",
            "url": f"data:image/jpeg;base64,{encode_image(image_path)}",
          },
        } for image_path in (images / dish).iterdir()
      ]
    }
  ],
  max_tokens=300,
)

In [23]:
response.choices[0].message.content

'Here are the ingredients for one person from the recipe in CSV format:\n\n```csv\ndish_name,name,tags,quantity,metric\nBacon Jalapeño Mac & Cheese,Bacon,Protein,2,unit\nBacon Jalapeño Mac & Cheese,Jalapeño,Vegetable,0.5,unit\nBacon Jalapeño Mac & Cheese,Scallions,Herb,1,unit\nBacon Jalapeño Mac & Cheese,Southwest Spice Blend,Spice,0.5,tbsp\nBacon Jalapeño Mac & Cheese,Garlic,Vegetable,1,clove\nBacon Jalapeño Mac & Cheese,Cavatappi Pasta,Grain,1,cup\nBacon Jalapeño Mac & Cheese,Panko Breadcrumbs,Grain,1, tbsp\nBacon Jalapeño Mac & Cheese,Flour,Grain,1,tbsp\nBacon Jalapeño Mac & Cheese,Cream Sauce Base,Dairy,0.5,cup\nBacon Jalapeño Mac & Cheese,Mexican Cheese Blend,Dairy,0.25,cup\nBacon Jalapeño Mac & Cheese,Monterey Jack,Dairy,0.25,cup\n```'